In [1]:
# 约瑟夫环问题
# 假设有n个人， 从第一个人开始报数， 数到m的人被淘汰
# person = ['Bob', 'Jack', 'Allen', 'Tony', 'Peter', 'Rose', 'Wade']


# 约瑟夫环是一个容器
# 容器一般具有的功能有 增 删 查 改
import copy
import csv
import zipfile

class Person:
    def __init__(self, name = '', age = -1):
        self.name = name
        self.age = age

class Reader:
    def __iter__(self):
        raise NotImplementedError

    def __next__(self):
        raise NotImplementedError

class TxtReader(Reader):
    def __init__(self, path):
        self.file = open(path)
       
    def __iter__(self):
        return self

    def __next__(self):
        row = self.file.readline()
        if not row:
            self.file.close()
            raise StopIteration
        item = row.strip().split(',')
        name, age = item[0], int(item[1])
        return Person(name, age)

class CSVReader(Reader):
    def __init__(self, path):
        self.csv_file = open(path)
        csv_reader = csv.reader(self.csv_file)
        self.it = csv_reader.__iter__()

    def __iter__(self):
        return self

    def __next__(self):
        item = self.it.__next__()
        if not item:
            self.csv_file.close()
            raise StopIteration
        name = item[0]
        age = item[1]
        return Person(name, age)

class TxtInZipReader(TxtReader):
    def __init__(self, path, target_file):
        with zipfile.ZipFile(path) as zip_file:
            target_path = zip_file.extract(target_file)
        TxtReader.__init__(self, target_path)

class CSVInZipReader(CSVReader):
    def __init__(self, path, target_file):
        with zipfile.ZipFile(path) as zip_file:
            target_path = zip_file.extract(target_file)
        CSVReader.__init__(self, target_path)

class JosephusRing:
    def __init__(self, reader = None):
        self.people = []
        self.start = -1
        self.step = -1
        if reader:
            for some_one in reader:
                self.people.append(some_one)

    # 增功能
    def append(self, obj):
        self.people.append(obj)

    # 删功能
    def pop(self, index):
        self.people.pop(index)
    
    # 查功能
    # 查找整个容器内容
    def query_list(self):
        temp = copy.copy(self.people)
        size = len(temp)
        if size == 0:
            return None
        
        index = self.start - 1
        ret = []
        for i in range(size):
            index = (index + self.step - 1) % len(temp)
            ret.append(temp.pop(index))
        return ret

    # 用生成器来进行一个一个输出 利用__next__
    def iter(self):
        temp = self.people
        start_index = self.start - 1
        while temp:    
            index = (start_index + self.step - 1) % len(temp)
            ret = temp.pop(index)
            start_index = index
            yield ret
        
        return None



In [2]:
csv_reader = CSVReader('person.csv')
jos = JosephusRing(csv_reader)
for each in jos.people:
    print(each.name, each.age)

print('-------------')

jos.start = 3
jos.step = 4
res = jos.query_list()
for each in res:
    print("name:{}, age:{}" .format(each.name, each.age))

Bob 15
Jack 12
Allen 17
Tony 20
Peter 15
Rose 16
Wade 19
-------------
name:Rose, age:16
name:Allen, age:17
name:Bob, age:15
name:Wade, age:19
name:Jack, age:12
name:Peter, age:15
name:Tony, age:20
